In [1]:
def getBaseMap(out_str,out_str2,road,end,wall,start_x,start_y,start_z,bound):
    return '''<?xml version="1.0" encoding="UTF-8" standalone="no" ?>
            <Mission xmlns="http://ProjectMalmo.microsoft.com" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
            
              <About>
                <Summary>Racing track simulation!</Summary>
              </About>
              
            <ServerSection>
              <ServerInitialConditions>
                <Time>
                    <StartTime>1000</StartTime>
                    <AllowPassageOfTime>false</AllowPassageOfTime>
                </Time>
                <Weather>clear</Weather>
              </ServerInitialConditions>
              <ServerHandlers>
                  <FlatWorldGenerator generatorString="3;7,2;1;"/>
                  <DrawingDecorator>
                    <DrawCuboid x1="-100"  y1="2" z1="-100"  x2="100" y2="20" z2="100" type="air" />
                    <DrawCuboid x1="-100"  y1="1" z1="-100"  x2="100" y2="1" z2="100" type="grass" />
                    {out_str}
                    {out_str2}
                    {road}
                    {end}
                    {wall}
                  </DrawingDecorator>
                  <ServerQuitFromTimeUp timeLimitMs="60000"/>
                  <ServerQuitWhenAnyAgentFinishes/>
                </ServerHandlers>
              </ServerSection>
              
              <AgentSection mode="Survival">
                <Name>Agent</Name>
                <AgentStart>
                    <Placement x="{start_x}" y="{start_y}" z="{start_z}" pitch="10" yaw="0"/>
                </AgentStart>
                <AgentHandlers>
                    <ObservationFromFullStats/>
                    <ContinuousMovementCommands/>
                    <RewardForTouchingBlockType>
                        <Block reward="1" type="packed_ice" behaviour="constant"/>
                        <Block reward="-10" type="log" behaviour="onceOnly"/>
                        <Block reward="-2" type="water" behaviour="onceOnly"/>
                        <Block reward="100" type="diamond_block" behaviour="onceOnly"/>
                    </RewardForTouchingBlockType>
                    <RewardForSendingCommand reward="-1" />
                    <AgentQuitFromTouchingBlockType>
                        <Block type="diamond_block" description="complete"/>
                        <Block type="log" description="fail"/>
                    </AgentQuitFromTouchingBlockType>
                    <ObservationFromGrid>
                        <Grid name="floorAll">
                            <min x="-4" y="1" z="0"/>
                            <max x="4" y="3" z="30"/>
                        </Grid>
                    </ObservationFromGrid>
                </AgentHandlers>
              </AgentSection>
            </Mission>'''.format(out_str = out_str,out_str2 = out_str2,road = road,end = end,wall= wall,start_x = start_x,start_y = start_y,start_z = start_z,bound = bound)


Remember to change reward blocks!!

In [2]:
import random
random.seed(1)
class Cfg:
    start_x,start_y,start_z = (1,2,0)
    # end_x,end_y,end_z = (-4,2,30)
    maze_bound = [-4,0,4,30] #x1, z1, x2,z2
    number_of_walls = 5
    end_block_type = "diamond_block"
    road_block_type = "packed_ice"
    wall_block_type = "log"
    water = "water"
    seed = 0

In [3]:
def save_maze_xml(num=20):
    for n in range(num):
        cfg = Cfg()
        road_str = '<DrawCuboid x1="{}"  y1="1" z1="{}"  x2="{}" y2="-1" z2="{}" type="{}" />'.format(cfg.maze_bound[0], cfg.maze_bound[1], cfg.maze_bound[2], cfg.maze_bound[3], cfg.road_block_type)
        destination_str = '<DrawCuboid x1="{}"  y1="-3" z1="{}"  x2="{}" y2="15" z2="{}" type="{}" />'.format(cfg.maze_bound[0], cfg.maze_bound[3], cfg.maze_bound[2], cfg.maze_bound[3] + 1, cfg.end_block_type)
        out_str = '<DrawCuboid x1="{}"  y1="3" z1="{}"  x2="{}" y2="1" z2="{}" type="{}" />'.format(cfg.maze_bound[0]-2, cfg.maze_bound[1]-2, cfg.maze_bound[2]+2, cfg.maze_bound[3]+2, cfg.wall_block_type)
        out_str2 = '<DrawCuboid x1="{}"  y1="3" z1="{}"  x2="{}" y2="2" z2="{}" type="air" />'.format(cfg.maze_bound[0]-1, cfg.maze_bound[1]-1, cfg.maze_bound[2]+1, cfg.maze_bound[3]+1)
        #<DrawLine x1="-4" y1="1" z1="13" x2="4" y2="1" z2="13" type="water"/> #line of water. change Z1 and Z2 for randomization.
        #between z -5 to z 25 
        block_list = ""
        for i in range(cfg.number_of_walls):
            while True:
                x, z = random.randint(cfg.maze_bound[0], cfg.maze_bound[2]), random.randint(cfg.maze_bound[1],
                                                                                            cfg.maze_bound[3])
                if abs(cfg.start_x - x) <= 3 and abs(cfg.start_z - z) <= 3:
                    continue
                break
            block_list += '<DrawCuboid x1="{}"  y1="0" z1="{}"  x2="{}" y2="3" z2="{}" type="{}" /> \n'.format(x, z, x, z, cfg.wall_block_type)
            
        block_list += '<DrawCuboid x1="{}"  y1="0" z1="{}"  x2="{}" y2="3" z2="{}" type="{}" /> \n'.format(cfg.start_x, cfg.start_z+3, cfg.start_x, cfg.start_z+3, cfg.wall_block_type)
        
        water_list = ""
        for i in range(cfg.number_of_walls):
            x1, x2 = random.randint(cfg.maze_bound[0] + 1, cfg.maze_bound[2] -1), random.randint(cfg.maze_bound[0]+ 1, cfg.maze_bound[2] - 1)
            z = random.randint(cfg.maze_bound[1]+2, cfg.maze_bound[3]- 3)
            water_list += '<DrawLine x1="{}" y1="1" z1="{}" x2="{}" y2="1" z2="{}" type="water"/> \n'.format(x1, z, x2, z)

        
        block_list += water_list
        #cfg.start_x, cfg.start_z = random.randint(cfg.maze_bound[0] + 1, cfg.maze_bound[2] - 1), random.randint(cfg.maze_bound[1] + 1, cfg.maze_bound[3] - 1)
        #dont want a random z start point.
        #cfg.start_x = random.randint(cfg.maze_bound[0] + 1, cfg.maze_bound[2] - 1)
        cfg.start_z = cfg.maze_bound[1] + 1
        xml_str = getBaseMap(out_str, out_str2, road_str, destination_str, block_list, cfg.start_x, cfg.start_y, cfg.start_z, cfg.maze_bound)
        with open("xmls/world_{}.txt".format(n), "w") as f:
            f.write(xml_str)

In [4]:

def load_maze_xml(i):
    xml_str = ""
    with open("xmls/world_{i}.txt".format(i), "r") as f:
        xml_str = f.read()
    return xml_str


In [5]:
save_maze_xml(25)